# Determining times for KLM Operators in our Calculator

In order to determine the values for KLM Operators in our calculator, we created a set of tasks that we executed while logging all important data, such as timestamps and pressed keys. The tasks were chosen to be of similar length, but with different numbers to prevent learning effects due to the participant memorizing the numbers.

The tasks were as follows:

```
Enter the following calculations using the keyboard:

55 * 12 - 100 =
/ 20 + 5 + 6 + 7 - 170 = 
10 * 30 - 150 =


Enter the following calculations using the mouse:

33 * 40 - 100 =
/ 30 + 8 + 9 + 1 - 170 =
22 * 10 - 230 =


Enter the following calculations using both mouse and keyboard. Switch the input method after every full 
number and operator (e.g. type "55 *" with the keyboard, then "33 =" with the mouse):

34 * 41 - 800 =
/ 60 + 4 + 1 + 3 - 650 = 
35 * 20 - 560 = 
```

We will analyze the logged data in the following paragraphs.

In [1]:
import pandas as pd
import statistics as stats
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('calc_logs.csv')
display(df)

,timestamp,key,input_type
0,2021-05-23 16:19:07.959645,5,KEYBOARD
1,2021-05-23 16:19:08.178622,5,KEYBOARD
2,2021-05-23 16:19:08.399618,*,KEYBOARD
3,2021-05-23 16:19:08.695590,1,KEYBOARD
4,2021-05-23 16:19:08.739606,2,KEYBOARD
...,...,...,...
97,2021-05-23 16:20:53.546660,-,KEYBOARD
98,2021-05-23 16:20:55.817579,5,BUTTON
99,2021-05-23 16:20:56.828606,6,BUTTON
100,2021-05-23 16:20:57.312667,0,BUTTON


In [3]:
# here we separate all inputs from the csv into the "blocks" in which they happened
# this is done by grouping all inputs with a delay of less than 5 seconds together

input_blocks = [] # list of lists - holds the separate chains of inputs
last_time = datetime.min
start_row = None
current_list = []

for row in df.iterrows():
    current_time = row[1]['timestamp']
    
    if start_row is None:
        start_row = row
        
    current_time = datetime.strptime(current_time, "%Y-%m-%d %H:%M:%S.%f")
        
    log_timedelta = current_time - last_time
    
    if log_timedelta.seconds > 5:
        if len(current_list) > 0:
            input_blocks.append(current_list.copy())
            current_list.clear()
            current_list.append(row)
        
    else:
        current_list.append(row)
        
    last_time = current_time
    
input_blocks.append(current_list.copy()) # add the list from the last block as well

# H - Operator

In [4]:
last_timestamp = None
last_row = None
hover_delta_list = []

for block in input_blocks:
    for row in block:
        if last_timestamp is None:
            last_timestamp = datetime.strptime(row[1]['timestamp'], "%Y-%m-%d %H:%M:%S.%f")
            last_row = row
            continue
            
        else:
            current_timestamp = datetime.strptime(row[1]['timestamp'], "%Y-%m-%d %H:%M:%S.%f")
            
            if last_row[1]['input_type'] != row[1]['input_type']:
                hover_delta_list.append((current_timestamp - last_timestamp).total_seconds() * 1000)
            
            last_timestamp = current_timestamp
    
    last_timestamp = None
    last_row = None
  
print("Hovering times between mouse and keyboard in milliseconds:")
print("Mean: " + str(stats.mean(hover_delta_list)))
print("Median: " + str(stats.median(hover_delta_list)))
print("Max: " + str(max(hover_delta_list)))
print("Min: " + str(min(hover_delta_list)))

Hovering times between mouse and keyboard in milliseconds:
Mean: 1409.42225
Median: 994.27
Max: 3921.998
Min: 484.06100000000004


# K - Operator

## Keyboard

In [5]:
keyboard_logs = input_blocks[0]
keyboard_logs_split = []

keyboard_k_delta_list = []

last_timestamp = None

# First, separate the keyboard inputs further into blocks for each task
# This is done in order to exclude the mental time required to read and process the new task block
temp_keyboard_list = []
for log in keyboard_logs:
    temp_keyboard_list.append(log)
    
    if log[1]['key'] == 'Evaluate':
        keyboard_logs_split.append(temp_keyboard_list.copy())
        temp_keyboard_list.clear()

# Write all time deltas between keypresses into a list
for block in keyboard_logs_split:
    for log in block:
        if last_timestamp is None:
            last_timestamp = datetime.strptime(log[1]['timestamp'], "%Y-%m-%d %H:%M:%S.%f")
            continue

        current_timestamp = datetime.strptime(log[1]['timestamp'], "%Y-%m-%d %H:%M:%S.%f")
        
        # Append to list as time in ms
        keyboard_k_delta_list.append((current_timestamp - last_timestamp).total_seconds() * 1000)

        last_timestamp = current_timestamp
    
    last_timestamp = None
    
print("Mean keyboard K in ms: " + str(stats.mean(keyboard_k_delta_list)))
print("Median keyboard K in ms: " + str(stats.median(keyboard_k_delta_list)))
print("Max keyboard K in ms: " + str(max(keyboard_k_delta_list)))
print("Min keyboard K in ms: " + str(min(keyboard_k_delta_list)))

Mean keyboard K in ms: 334.3808
Median keyboard K in ms: 248.72
Max keyboard K in ms: 1494.9920000000002
Min keyboard K in ms: 44.016


# P and B Operators

Since our calculator can only measure times between button presses, we cannot measure P (pointing) and B (mouse button press/release) separately. In our tasks every P is followed by BB (button down and button up). We are assuming B to be at 100ms and calculate P dependant on that.

In [6]:
mouse_logs = input_blocks[1].copy()
mouse_logs_split = []

mouse_delta_list = []

last_timestamp = None

B_TIME = 100
BB_TIME = 200  # 100ms time for button down / button up respectively
bb_delta = timedelta(milliseconds=BB_TIME)

temp_mouse_list = []
last_key = ""
skip_counter = 0

# First, separate the mouse inputs further into blocks for each task
# This is done in order to exclude the mental time required to read and process the new task block
# Additionally, if the same key is pressed multiple times in a row, the time is excluded, since no further
# pointing is required.
# In this case, the time needed for the mouse click (BB) is "refunded"
for log in mouse_logs:
    log_copy = log  # use a copy of the tuple in order to not change the original
    log = None
    if last_key == log_copy[1]['key']:
        skip_counter += 1
        continue
        
    adjusted_timestamp = datetime.strptime(log_copy[1]['timestamp'], "%Y-%m-%d %H:%M:%S.%f") 
    - (skip_counter * bb_delta)
    log_copy[1]['timestamp'] = adjusted_timestamp
    temp_mouse_list.append(log_copy)
    last_key = log_copy[1]['key']
    skip_counter = 0
    
    if log_copy[1]['key'] == 'Evaluate':
        mouse_logs_split.append(temp_mouse_list.copy())
        temp_mouse_list.clear()

# Write all time deltas between keypresses into a list
for block in mouse_logs_split:
    for log in block:
        if last_timestamp is None:
            last_timestamp = log[1]['timestamp']
            continue

        current_timestamp = log[1]['timestamp']
        
        # Append to list as time in ms
        mouse_delta_list.append((current_timestamp - last_timestamp).total_seconds() * 1000 - BB_TIME)

        last_timestamp = current_timestamp
    
    last_timestamp = None
    
print("Mean P in ms: " + str(stats.mean(mouse_delta_list)))
print("Median P in ms: " + str(stats.median(mouse_delta_list)))
print("Max P in ms: " + str(max(mouse_delta_list)))
print("Min P in ms: " + str(min(mouse_delta_list)))

Mean P in ms: 585.3735357142857
Median P in ms: 452.18850000000003
Max P in ms: 1532.8220000000001
Min P in ms: 136.015


# M - Operator

For the M - Operator, we assume the time needed for reading and processing a new input block from our task list. This can be determined by looking at the time difference between pressing the "=" button (end of an input block) and the next button press (beginning of a new input block)

In [7]:
evaluate_timestamp = None
next_block_timestamp = None

m_delta_list = []

for block in input_blocks:
    for log in block:
        if evaluate_timestamp is not None:
            
            if isinstance(log[1]['timestamp'], datetime):
                m_delta_list.append((log[1]['timestamp']
                               - datetime.strptime(evaluate_timestamp, "%Y-%m-%d %H:%M:%S.%f"))
                                    .total_seconds() * 1000)
            else:
                m_delta_list.append((datetime.strptime(log[1]['timestamp'], "%Y-%m-%d %H:%M:%S.%f")
                                   - datetime.strptime(evaluate_timestamp, "%Y-%m-%d %H:%M:%S.%f"))
                                    .total_seconds() * 1000)
            evaluate_timestamp = None
            
        if log[1]['key'] == 'Evaluate':
            evaluate_timestamp = str(log[1]['timestamp'])
            
print("Mean M in ms: " + str(stats.mean(m_delta_list)))
print("Median M in ms: " + str(stats.median(m_delta_list)))
print("Max M in ms: " + str(max(m_delta_list)))
print("Min M in ms: " + str(min(m_delta_list)))
            
    

Mean M in ms: 5536.665625000001
Median M in ms: 2657.5905000000002
Max M in ms: 16696.469
Min M in ms: 831.923


# Summary

In [8]:
print("All times are given in ms\n")
print("K: " + str(stats.median(keyboard_k_delta_list)))
print("P: " + str(stats.median(mouse_delta_list)))
print("H: " + str(stats.median(hover_delta_list)))
print("M: " + str(stats.median(m_delta_list)))
print("B: " + str(B_TIME) + " (assumed as constant)")
print("D: not applicable" )

All times are given in ms

K: 248.72
P: 452.18850000000003
H: 994.27
M: 2657.5905000000002
B: 100 (assumed as constant)
D: not applicable
